In [1]:
import numpy as np

def g(player):
    phi = player.phi
    return 1/(np.sqrt(1+3*(phi/(np.pi))**2))

def E(player1,player2):
    mu = player1.mu
    mu2 = player2.mu
    return 1/(1+np.exp(g(player2)*(mu2-mu)))

class Player:
    tau = 1/np.sqrt(2) #this is a constant that can be fine-tuned.
    def __init__(self, name, mu=0, phi=2, rho=0.06):
        self.name = name
        self.mu = mu
        self.phi = phi
        self.rho = rho
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return self.name == other.name
        else:
            return False

    def estimate_variance(self, games):
        v_sum = 0
        for game in games:
            if self == game.player1:
                E_iter = E(self,game.player2)
                v_sum = v_sum + (g(game.player2))**2*(E_iter-E_iter**2)
            if self == game.player2:
                E_iter = E(self,game.player1)
                v_sum = v_sum + (g(game.player1))**2*(E_iter-E_iter**2)
        self.est_variance = 1/v_sum

    def estimate_Delta(self, games):
        d_sum = 0
        for game in games:
            if self == game.player1:
                d_sum = d_sum + g(game.player2)*(game.score-E(self,game.player2))
            if self == game.player2:
                d_sum = d_sum + g(game.player1)*(1-game.score-E(self,game.player1))
        self.est_Delta = self.est_variance*d_sum
    
    def _f(self,x):
        ex = np.exp(x)
        a = np.log(self.phi**2)
        return (ex*(self.est_Delta**2-self.phi**2-self.est_variance-ex)/(2*(self.phi**2+self.est_variance+ex)**2)
                - (x - a)/self.tau**2)
    
    def compute_volatility(self):
        #basically the Illinois algorithm lifted from the glicko2 pdf. Illini!
        a = np.log(self.phi**2)
        A = a
        if self.est_Delta**2>(self.phi**2+self.est_variance):
            B = np.log(self.est_Delta**2-self.phi**2-self.est_variance)
        else:
            k = 1
            while(self._f(a-k*self.tau)<0):
                k = k+1
            B = a-k*self.tau
        fA = self._f(A)
        fB = self._f(B)
        while(abs(B-A)>1e-6):
            C = A + (A-B)*fA/(fB-fA)
            fC = self._f(C)
            if fC*fB<0:
                A = B
                fA = fB
            else:
                fA = fA/2    
            B = C
            fB = fC
        self.rho = np.exp(A/2)
        phi_d = np.sqrt(self.phi**2+self.rho**2)
        self.phi = 1/np.sqrt(1/phi_d**2+1/self.est_variance)
        self.mu = self.mu + self.phi**2/self.est_variance*self.est_Delta

class Game:
    def __init__(self, player1, player2, score):
        '''score=1 means player1 wins, score=0 means player2 wins, score=0.5 means draw'''
        self.player1 = player1
        self.player2 = player2
        self.score = score

        

In [2]:
BOS = Player('BOS')
FLA = Player('FLA')
HOU = Player('HOU')
LDN = Player('LDN')
NYE = Player('NYE')
PHI = Player('PHI')
DAL = Player('DAL')
GLA = Player('GLA')
VAL = Player('VAL')
SFS = Player('SFS')
SEO = Player('SEO')
SHD = Player('SHD')
games = [
    Game(SFS,VAL,0), Game(SFS,VAL,0), Game(SFS,VAL,0), Game(SFS,VAL,0), 
    Game(SHD,GLA,0), Game(SHD,GLA,0), Game(SHD,GLA,0), Game(SHD,GLA,0), 
    Game(DAL,SEO,1), Game(DAL,SEO,0), Game(DAL,SEO,0), Game(DAL,SEO,0.5), 
    Game(LDN,FLA,0), Game(LDN,FLA,1), Game(LDN,FLA,1), Game(LDN,FLA,1), 
    Game(PHI,HOU,1), Game(PHI,HOU,0), Game(PHI,HOU,1), Game(PHI,HOU,0), Game(PHI,HOU,1), 
    Game(BOS,NYE,0), Game(BOS,NYE,1), Game(BOS,NYE,0), Game(BOS,NYE,0), 
    Game(VAL,DAL,1), Game(VAL,DAL,1), Game(VAL,DAL,1), Game(VAL,DAL,0.5), 
    Game(FLA,BOS,0), Game(FLA,BOS,0), Game(FLA,BOS,0), Game(FLA,BOS,0), 
    Game(SFS,SHD,1), Game(SFS,SHD,1), Game(SFS,SHD,1), Game(SFS,SHD,0), 
    Game(LDN,PHI,1), Game(LDN,PHI,1), Game(LDN,PHI,1), Game(LDN,PHI,1), 
    Game(NYE,HOU,1), Game(NYE,HOU,1), Game(NYE,HOU,1), Game(NYE,HOU,0), 
    Game(SEO,GLA,1), Game(SEO,GLA,1), Game(SEO,GLA,1), Game(SEO,GLA,1), 
]
players = [BOS, FLA, HOU, LDN, NYE, PHI, DAL, GLA, VAL, SFS, SEO, SHD]

In [3]:
for player in players:
    player.estimate_variance(games)
    player.estimate_Delta(games)
    player.compute_volatility()

for player in players:
    print('{0}: {1}±{2}'.format(player.name,player.mu,2*player.phi))

BOS: 0.6499074053584527±1.9671661101873676
FLA: -1.5001163356737428±1.758280405416066
HOU: -0.8791062776127051±1.868057982953958
LDN: 1.0660724734659788±1.8971634986663597
NYE: 1.0464797183403194±1.6196987428445264
PHI: -0.8509203850644641±1.5735262926889682
DAL: -1.3012830479991642±1.9682768266648336
GLA: -1.0736673394958162e-16±1.9667979723547062
VAL: 1.5289127539159433±1.867472991086238
SFS: -0.05009400201455155±1.9096112327235935
SEO: 0.8770079409843778±1.6495084286525703
SHD: -1.5919337555188±1.540770740048951


In [4]:
for player in players:
    print('{0}: {1}±{2}'.format(player.name,player.mu*173.7178+1500,2*player.phi*173.7178))

BOS: 1612.9004846625787±341.73176889630713
FLA: 1239.4030904226959±305.44460381198707
HOU: 1347.2835914869315±324.5149230711991
LDN: 1685.1957647310683±329.57106922862295
NYE: 1681.7921544147±281.3705022697169
PHI: 1352.1799827314485±273.3495258080836
DAL: 1273.9439717242908±341.92472011919625
GLA: 1500.0±341.6678168019204
VAL: 1765.599360002219±324.4132995709209
SFS: 1491.2977801768366±331.7334622040307
SEO: 1652.351890090336±286.5489753069815
SHD: 1223.4527702455362±267.65930326567565


In [5]:
print('SFS v PHI: {}'.format(8*(E(SFS,PHI)-0.5)))
print('FLA v SEO: {}'.format(8*(E(FLA,SEO)-0.5)))
print('HOU v SHD: {}'.format(8*(E(HOU,SHD)-0.5)))

SFS v PHI: 1.4066635534171894
FLA v SEO: -3.1756987936816437
HOU v SHD: 1.2670693402868913
